一、项目背景
本数据报告以淘宝app平台为数据集，通过行业的指标对淘宝用户行为进行分析，从而探索淘宝用户的行为模式，具体指标包括：日PV和日UV分析，付费率分析，复购行为分析，漏斗流失分析和用户价值RFM分析。

二、数据来源
https://tianchi.aliyun.com/dataset/dataDetail?dataId=46&userId=1

三、分析目标

1.日PV有多少

2.日UV有多少

3.付费率情况如何

4.复购率是多少

5漏斗流失情况如何

6.用户价值情况

四、理解数据
本数据集共有104万条左右数据，数据为淘宝APP2014年11月18日至2014年12月18日的用户行为数据，共计6列字段，列字段分别是：

user_id：用户身份，脱敏

item_id：商品ID，脱敏

behavior_type：用户行为类型（包含点击、收藏、加购物车、支付四种行为，分别用数字1、2、3、4表示）

user_geohash：地理位置

item_category：品类ID（商品所属的品类）

time：用户行为发生的时间



In [5]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns
data_user = pd.read_csv('./input/tianchi_mobile_recommend_train_user.csv')
data_user.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21


In [16]:
#缺失值处理
missingTotal = data_user.isnull().sum()
missingExist = missingTotal[missingTotal>0]
missingExist = missingExist.sort_values(ascending=False)

In [19]:
print(missingExist)

user_geohash    8334824
dtype: int64


In [23]:
print(missingTotal)

user_id                0
item_id                0
behavior_type          0
user_geohash     8334824
item_category          0
time                   0
dtype: int64


存在缺失值的是User_geohash，有8334824条，不能删除缺失值，因为地理信息在数据集收集过程中做过加密转换，因此对数据集不做处理。

In [24]:
#一致化处理
import re
#拆分数据集
data_user['date'] = data_user['time'].map(lambda s: re.compile(' ').split(s)[0])

In [25]:
data_user['hour']=data_user['time'].map(lambda s:re.compile(' ').split(s)[1])
data_user.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time,date,hour
0,98047837,232431562,1,NaN,4245,2014-12-06 02,2014-12-06,02
1,97726136,383583590,1,NaN,5894,2014-12-09 20,2014-12-09,20
2,98607707,64749712,1,NaN,2883,2014-12-18 11,2014-12-18,11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10,2014-12-06,10
4,98145908,290208520,1,NaN,13926,2014-12-16 21,2014-12-16,21


In [26]:
#查看data_user数据集数据类型：
data_user.dtypes

user_id           int64
item_id           int64
behavior_type     int64
user_geohash     object
item_category     int64
time             object
date             object
hour             object
dtype: object

In [27]:
data_user['date']=pd.to_datetime(data_user['date'])
data_user['time']=pd.to_datetime(data_user['time'])
data_user['hour']=data_user['hour'].astype('int64')
data_user.dtypes

user_id                   int64
item_id                   int64
behavior_type             int64
user_geohash             object
item_category             int64
time             datetime64[ns]
date             datetime64[ns]
hour                      int64
dtype: object

In [28]:
#异常值处理
data_user = data_user.sort_values(by='time',ascending=True)
data_user = data_user.reset_index(drop=True)
# pandas 的 describe 方法用于生成描述性统计信息的汇总表，主要用于数据框或系列（Series）。它能够对数值型数据、类别型数据和混合数据类型生成统计摘要。
data_user.describe()

,user_id,item_id,behavior_type,item_category,time,date,hour
count,1.225691e+07,1.225691e+07,1.225691e+07,1.225691e+07,12256906,12256906,1.225691e+07
mean,7.170732e+07,2.023084e+08,1.105271e+00,6.846162e+03,2014-12-04 04:47:28.445702144,2014-12-03 13:58:23.675396352,1.481799e+01
min,4.913000e+03,6.400000e+01,1.000000e+00,2.000000e+00,2014-11-18 00:00:00,2014-11-18 00:00:00,0.000000e+00
25%,3.584965e+07,1.014130e+08,1.000000e+00,3.721000e+03,2014-11-26 15:00:00,2014-11-26 00:00:00,1.000000e+01
50%,7.292804e+07,2.021359e+08,1.000000e+00,6.209000e+03,2014-12-04 14:00:00,2014-12-04 00:00:00,1.600000e+01
75%,1.073774e+08,3.035405e+08,1.000000e+00,1.029000e+04,2014-12-11 23:00:00,2014-12-11 00:00:00,2.000000e+01
max,1.424559e+08,4.045625e+08,4.000000e+00,1.408000e+04,2014-12-18 23:00:00,2014-12-18 00:00:00,2.300000e+01
std,4.122920e+07,1.167397e+08,4.572662e-01,3.809922e+03,NaN,NaN,6.474778e+00


通过观察数据集的四分位数，总数，平均值，方差等，发现数据集并无异常值存在。